**XGBoost(X(Extreme GBoost))**, 高效的实现了GBDT算法，并进行了算法和工程上的改进。  
* XGBoost是一个优化的分布式梯度增强库，旨在实现高效、灵活和便携。  
* XGBboost可以在分布式环境运行，支持大数据样本训练。  
* 以CART树为基模型。  

**XGBoost与GBDT的不同**：  
* 1，GBDT是机器学习算法，XGBoost是该算法的工程实现  
* 2，XGBoost（CART）显示地加入了正则项来控制模型的复杂度，防止过拟合
* 3，XGBoost 支持多种类型的基分类器，且在每轮迭代时，支持样本采样
* 4，XGBoost能够自动学习出缺失值的处理策略
* 5，XGBoost使用泰勒展开，优化了目标函数

**XGBoost的核心算法思想：**  

    1，不断添加树，不断地进行特征分裂来生长一棵树，每次添加一棵树，其实是学习一个新函数去拟合上次预测的残差。  
    2，当我们训练完成得到K棵树，我们要预测一个样本的分数，其实就是根据这个样本的特征，在每棵树中会落到对应的一个叶子节点，每个叶子节点就对应一个分数。  
    3，最后，只需要将每棵树对应的分数加起来就是该样本的预测值。

**XGBoost代码演示**

In [1]:
import xgboost as xgb

In [2]:
# read in data
dtrain = xgb.DMatrix('datasets/agaricus/agaricus.txt.train')
dtest = xgb.DMatrix('datasets/agaricus/agaricus.txt.test')
# specify parameters via map
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

[00:07:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [3]:
dtest.num_row(),preds.shape

(1611, (1611,))

In [4]:
preds

array([0.28583017, 0.9239239 , 0.28583017, ..., 0.9239239 , 0.05169873,
       0.9239239 ], dtype=float32)

**XGBoost 参数**  
* 通用参数
 * Tree Booster 参数
 * Linear Booster 参数
* 学习任务参数
* 命令行参数

**1，通用参数**  
* booster: 基模型，默认gbtree, 可选[gbtree, gblinear, dart]
* nthread: 并行的线程数
* verbosity: 打印消息的详细程度，有效值为0（静默）、1（警告）、2（信息）、3（调试）
* validate_parameters：验证参数，是否对输入参数进行验证

**1.1 Tree Booster参数**  
* eta: learning_rate, 学习率，默认0.3
* gamma: min_split_loss, 划分节点所需要的最小损失值，默认0，值越大，越不容易过拟合
* max_depth：树的最大深度，默认6
* min_child_weight: 默认1，划分节点的样本权重和小于该值，停止分裂
* max_delta_step: 我们允许每个叶输出的最大增量步长。如果该值设置为0，则表示没有约束。如果将其设置为正值，则有助于使更新步骤更加保守。通常不需要这个参数，但当类别极度不平衡时，它可能有助于逻辑回归。将其设置为1-10可能有助于控制更新。
* subsample: 构建每棵树时对样本的采样率，默认1
* sampling_method: 采样方法，默认uniform，可选[uniform, gradient_based]
* colsample_bytree：列采样率，即特征采样率，默认1
* lambda：L2正则化权重，默认1，增加此值会使模型更保守
* alpha：默认= 0，权重的L1正则化项。增加此值将使模型更加保守。
* tree_method：默认=auto，XGBoost中使用的树构建算法。
  * auto：使用启发式选择最快的方法。
  * exact：精确的贪婪算法。枚举所有拆分的候选点。
  * approx：使用分位数和梯度直方图的近似贪婪算法。
  * hist：更快的直方图优化的近似贪婪算法。（LightGBM也是使用直方图算法）
  * gpu_hist：GPU hist算法的实现。
* scale_pos_weight:控制正负权重的平衡，这对于不平衡的类别很有用。Kaggle竞赛一般设置sum(negative instances) / sum(positive instances)，在类别高度不平衡的情况下，将参数设置大于0，可以加快收敛。
* num_parallel_tree：默认=1，每次迭代期间构造的并行树的数量。此选项用于支持增强型随机森林。
* monotone_constraints：可变单调性的约束，在某些情况下，如果有非常强烈的先验信念认为真实的关系具有一定的质量，则可以使用约束条件来提高模型的预测性能。（例如* params_constrained\['monotone_constraints'\] = "(1,-1)"，(1,-1)我们告诉XGBoost对第一个预测变量施加增加的约束，对第二个预测变量施加减小的约束。）

**1.2 Linear Booster参数**   
* lambda（reg_lambda）：默认= 0，L2正则化权重项。增加此值将使模型更加保守。归一化为训练示例数。
* alpha（reg_alpha）：默认= 0，权重的L1正则化项。增加此值将使模型更加保守。归一化为训练示例数。
* updater：默认= shotgun。
    * shotgun：基于shotgun算法的平行坐标下降算法。使用“ hogwild”并行性，因此每次运行都产生不确定的解决方案。
    * coord_descent：普通坐标下降算法。同样是多线程的，但仍会产生确定性的解决方案。
* feature_selector：默认= cyclic。特征选择和排序方法
    * cyclic：通过每次循环一个特征来实现的。
    * shuffle：类似于cyclic，但是在每次更新之前都有随机的特征变换。
    * random：一个随机(有放回)特征选择器。
    * greedy：选择梯度最大的特征。（贪婪选择）
    * thrifty：近似贪婪特征选择（近似于greedy）
    * top_k：要选择的最重要特征数（在greedy和thrifty内）

**2. 任务参数**  
* objective：默认为reg:squarederror，表示最小平方误差
 * reg:squarederror：最小平方误差
 * reg:squaredlogerror：对数平方损失。$\frac{1}{2}[log(pred+1)-log(label+1)]^2$
 * reg:logistic：逻辑回归
 * reg:pseudohubererror：使用伪Huber损失进行回归，这是绝对损失的两倍可微选择
 * binary:logistic：二元分类的逻辑回归，输出概率
 * binary:logitraw：用于二进制分类的逻辑回归，逻辑转换之前的输出得分
 * binary:hinge：二进制分类的铰链损失，这使预测为0或1，而不是产生概率（SVM就是铰链损失函数）
 * count:poisson：统计数据的泊松回归，泊松分布的输出平均值
 * survival:cox：针对正确的生存时间数据进行Cox回归（负值被视为正确的生存时间）
 * survival:aft：用于检查生存时间数据的加速故障时间模型
 * aft_loss_distribution：survival:aft和aft-nloglik度量标准使用的概率密度函数
 * multi:softmax：设置XGBoost以使用softmax目标进行多类分类，还需要设置num_class（类数）
 * multi:softprob：与softmax相同，但输出向量，可以进一步重构为矩阵，结果包含属于每个类别的每个数据点的预测概率。
 * rank:pairwise：使用LambdaMART进行成对排序，从而使成对损失最小化
 * rank:ndcg：使用LambdaMART进行列表式排序，使标准化折让累积收益（NDCG）最大化
 * rank:map：使用LambdaMART进行列表平均排序，使平均平均精度（MAP）最大化
 * reg:gamma：使用对数链接进行伽马回归，输出是伽马分布的平均值
 * reg:tweedie：使用对数链接进行Tweedie回归
 * 自定义损失函数和评价指标：https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html
* eval_metric：验证数据的评估指标，将根据目标分配默认指标（回归均方根，分类误差，排名的平均平均精度），用户可以添加多个评估指标
 * rmse：均方根误差；rmsle：均方根对数误差；mae：平均绝对误差；mphe：平均伪Huber错误； * logloss：负对数似然；error：二进制分类错误率；
 * merror：多类分类错误率；mlogloss：多类logloss；auc：曲线下面积；aucpr：PR曲线下的面积；ndcg：归一化累计折扣；map：平均精度；
* seed ：随机数种子，[默认= 0]。

**XGBoost一般调参步骤**  

   - 1. 确定学习速率和提升参数调优的初始值

   - 2. max_depth 和 min_child_weight 参数调优

   - 3. gamma(划分节点的最小损失值)参数调优

   - 4. 采样率：subsample 和 colsample_bytree 参数优

   - 5. 正则化参数alpha调优

   - 6. 降低学习速率和使用更多的决策树

**XGBoost 鸢尾花分类**

In [5]:
# 导入库
import xgboost as xgb
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [6]:
# 加载数据
iris = load_iris()
X,y = iris.data, iris.target
col = iris.target_names 
# 划分训练集和验证
train_x, valid_x, train_y, valid_y = train_test_split(X, y, 
                                                      test_size=0.3, 
                                                      random_state=1)

In [13]:
train_x, train_y

(array([[7.7, 2.6, 6.9, 2.3],
        [5.7, 3.8, 1.7, 0.3],
        [5. , 3.6, 1.4, 0.2],
        [4.8, 3. , 1.4, 0.3],
        [5.2, 2.7, 3.9, 1.4],
        [5.1, 3.4, 1.5, 0.2],
        [5.5, 3.5, 1.3, 0.2],
        [7.7, 3.8, 6.7, 2.2],
        [6.9, 3.1, 5.4, 2.1],
        [7.3, 2.9, 6.3, 1.8],
        [6.4, 2.8, 5.6, 2.2],
        [6.2, 2.8, 4.8, 1.8],
        [6. , 3.4, 4.5, 1.6],
        [7.7, 2.8, 6.7, 2. ],
        [5.7, 3. , 4.2, 1.2],
        [4.8, 3.4, 1.6, 0.2],
        [5.7, 2.5, 5. , 2. ],
        [6.3, 2.7, 4.9, 1.8],
        [4.8, 3. , 1.4, 0.1],
        [4.7, 3.2, 1.3, 0.2],
        [6.5, 3. , 5.8, 2.2],
        [4.6, 3.4, 1.4, 0.3],
        [6.1, 3. , 4.9, 1.8],
        [6.5, 3.2, 5.1, 2. ],
        [6.7, 3.1, 4.4, 1.4],
        [5.7, 2.8, 4.5, 1.3],
        [6.7, 3.3, 5.7, 2.5],
        [6. , 3. , 4.8, 1.8],
        [5.1, 3.8, 1.6, 0.2],
        [6. , 2.2, 4. , 1. ],
        [6.4, 2.9, 4.3, 1.3],
        [6.5, 3. , 5.5, 1.8],
        [5. , 2.3, 3.3, 1. ],
        [6

In [11]:
# 训练
parameters = {
              'max_depth': [5, 10, 15, 20, 25],
              'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15],
              'n_estimators': [500, 1000, 2000, 3000, 5000],
              'min_child_weight': [0, 2, 5, 10, 20],
              'max_delta_step': [0, 0.2, 0.6, 1, 2],
              'subsample': [0.6, 0.7, 0.8, 0.85, 0.95],
              'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9],
              'reg_alpha': [0, 0.25, 0.5, 0.75, 1],
              'reg_lambda': [0.2, 0.4, 0.6, 0.8, 1],
              'scale_pos_weight': [0.2, 0.4, 0.6, 0.8, 1]

}

xlf = xgb.XGBClassifier(max_depth=10,
            learning_rate=0.01,
            n_estimators=200,
            silent=True,
            objective='multi:softmax',
            num_class=3 ,          
            nthread=-1,
            gamma=0,
            min_child_weight=1,
            max_delta_step=0,
            subsample=0.85,
            colsample_bytree=0.7,
            colsample_bylevel=1,
            reg_alpha=0,
            reg_lambda=1,
            scale_pos_weight=1,
            seed=0,
            missing=None,
            verbosity=0)

# 网格调参
gs = GridSearchCV(xlf, param_grid=parameters, scoring='accuracy', cv=3)
gs.fit(train_x, train_y)

c:\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\python39\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "c:\python39\li

c:\python39\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", line 1209, in predict
    class_probs = super().predict(
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", li

c:\python39\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", line 1209, in predict
    class_probs = super().predict(
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", li

c:\python39\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", line 1209, in predict
    class_probs = super().predict(
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", li

c:\python39\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", line 1209, in predict
    class_probs = super().predict(
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", li

...

c:\python39\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", line 1209, in predict
    class_probs = super().predict(
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", li

c:\python39\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", line 1209, in predict
    class_probs = super().predict(
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", li

c:\python39\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", line 1209, in predict
    class_probs = super().predict(
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", li

c:\python39\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\python39\lib\site-packages\sklearn\metrics\_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", line 1209, in predict
    class_probs = super().predict(
  File "c:\python39\lib\site-packages\xgboost\sklearn.py", li

KeyboardInterrupt: 

In [10]:
print("Best score: %0.3f" % gs.best_score_)
print("Best parameters set: %s" % gs.best_params_ )

Best score: nan
Best parameters set: {'max_depth': 5}
